In [ ]:

import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 



In [7]:
current_dir
#parent_dir

'C:\\Users\\josel\\AppData\\Local\\Temp\\ipykernel_20988'

In [ ]:
# Now you can import a module from the parent directory
from agents import *

import random

random.seed(1234)

In [ ]:
collections
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

In [ ]:
# Agent Class - Fire brigade Agent - Model
class ModelFireman(Agent):
    location = [0, 0]
    Name = 'ModelFireman'
    Agent.isHoldingWater = False

    def takeWater(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

    def estinguishFire(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Fire):
            return True
        return False

In [ ]:
# model = {(x, y): None for x in range(6) for y in range(6)}

def programAgent(percept):
    location, things = percept
    for t in things:
        # if model[tuple(location)] not in['Visited']:
        if isinstance(t, Fire):
            model[tuple(location)] = 'Fire'
            print('There is Fire')
            return 'Estinguish Fire'
        elif isinstance(t, Water):
            model[tuple(location)] = 'Water'
            print('There is Water')
            return 'Take Water'
        model[tuple(location)] = t  # Update the model here

    if not isinstance(model[tuple(location)], (Water, Fire)):
        model[tuple(location)] = 'Visited'

    choice = random.choice(('L', 'R', 'D', 'U'))
    if choice !=0:
        if choice == 'L':
            pass
        elif choice == 'R':
            pass
        elif choice == 'D':
            pass
        elif choice == 'U':
            pass
    return 'Move' + choice

    return programAgent  # , model

In [ ]:
########################################
# THING CLASSES
########################################
# Thing Class - Water
class Water(Thing):
    Name='Water'
    pass

# Thing Class - Fire


class Fire(Thing):
    Name='Fire'
    pass


class Wall(Obstacle):
    Name='Wall'
    pass


class Path(Thing):
    pass

In [ ]:
class Forrest2D(GraphicEnvironment):
    def percept(self, agent):
        listThings = []
        for things in self.list_things_at(agent.location):
            if things != agent:
                listThings.append(things)
        """By default, agent perceives things within a default radius."""
        return agent.location, listThings

    def thing_classes(self):
        return [Fire, Water, ModelFireman]

    def execute_action(self, agent, action):
        if action == 'MoveR':
            if agent.location[0]>=0 and agent.location[0]< 5:
                agent.location[0]+=1
                agent.performance -= 1
                # print('R here we go')
        elif action == 'MoveL':
            if agent.location[0]>0 and agent.location[0]<=5:
                agent.location[0]-=1
                agent.performance -= 1
                # print('L here we go')
        elif action == 'MoveU':
            if agent.location[1]>=0 and agent.location[1]<5:
                agent.location[1]+=1
                agent.performance -= 1
                # print('U here we go')
        elif action == 'MoveD':
            if agent.location[1]>0 and agent.location[1]<=5:
                agent.location[1]-=1
                agent.performance -= 1
                # print('D here we go')
        elif action == 'Take Water':
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.takeWater(items[0]):
                    agent.performance += 100
                    self.delete_thing(items[0])
                    agent.isHoldingWater = True
                    print('New action to Take Water')
        elif action == 'Estinguish Fire':
            items = self.list_things_at(agent.location, tclass=Fire)
            if len(items) != 0:
                if agent.estinguishFire(items[0]) and agent.isHoldingWater:
                    agent.performance += 100
                    self.delete_thing(items[0])
                    agent.isHoldingWater = False
                    print('New action to Estinguish Fire')
                else:
                    agent.performance -= 100
                    print('Burning, no water')

            if model[tuple(agent.location)] == 'Visited':
                agent.performance -= 10

        print(model[tuple(agent.location)])
        print(agent.location)
        print(
            f'Fireman performance: {agent.performance} and is holding water: {agent.isHoldingWater}')

    def is_done(self):

        return False

In [ ]:
model = {(x, y): None for x in range(width) for y in range(height)}
forrest = Forrest2D(width, height, color={'ModelFireman': (230, 115, 40), 'Water': (
    0, 200, 200), 'Fire': (200, 0, 0), 'Wall': (50, 50, 50), 'Path': (120, 0, 100)})

In [ ]:
# Define the positions of obstacles.
obstacles = [(5, 4), (4, 4), (2, 4), (1, 4), (3, 4), (6, 4), (7, 4), (8, 4), (9, 4)]


def add_obstacles(self, obstacles):
    for each in obstacles:
        print(each)
        x, y = each
        self.add_thing(Wall(), (x, y))

In [ ]:
fireman = ModelFireman(programAgent)
fire = Fire()
water = Water()
wall = Wall()

forrest.add_thing(fireman, [1, 1])
forrest.add_thing(water, [3, 4])
forrest.add_thing(fire, [3, 5])
add_obstacles(forrest, obstacles)

In [ ]:
forrest.run(500, delay=0.01)

In [ ]:
print(f'Fireman performance: {fireman.performance} and is holding water: {fireman.isHoldingWater}')

In [ ]:
print(model)

In [ ]:
class FirefightingProblem(Problem):
    def __init__(self, initial, goal=None, graph=None):
        super().__init__(initial, goal)
        self.graph = graph

    def actions(self, state):
        # Return possible movements (up, down, left, right) that do not lead to obstacles.
        x, y = state
        possible_actions = []

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = x + dx, y + dy
            if self.graph[new_x][new_y] != 'Obstacle':
                possible_actions.append((new_x, new_y))

        return possible_actions

    def result(self, state, action):
        # Return the state after taking the given action.
        return action

    def goal_test(self, state):
        # Check if the state is at the exit point (goal).
        return state == self.goal

    def path_cost(self, cost_so_far, state1, action, state2):
        # Assign a cost of 1 per move (action).
        return cost_so_far + 1

    def h(self, node):
        # Estimate the remaining distance to the goal using Euclidean distance.
        x1, y1 = node.state
        x2, y2 = self.goal
        return int(distance.euclidean((x1, y1), (x2, y2)))

In [ ]:
class FirefightingNode(Node):
    def __init__(self, state, parent=None, action=None, path_cost=0):
        super().__init__(state, parent, action, path_cost)
        self.depth = 0 if parent is None else parent.depth + 1

In [ ]:
def create_firefighting_graph(width, height, obstacles, start, goal):
    # Create a grid-based graph representing the firefighting environment.
    graph = [[''] * height for _ in range(width)]

    for x, y in obstacles:
        graph[x][y] = 'Obstacle'

    graph[start[0]][start[1]] = 'Start'
    graph[goal[0]][goal[1]] = 'Goal'

    return graph

In [ ]:
# Define the dimensions of the grid and the initial state.
start = (1, 1)
goal = (6, 7)

In [ ]:
# Create the grid-based graph.
graph = create_firefighting_graph(width, height, obstacles, start, goal)

# Create a FirefightingProblem instance with the defined initial and goal states.
problem = FirefightingProblem(start, goal, graph)

# Find the optimal path using A* search.
# goal_node = astar_search(problem, display=True)

goal_node = best_first_graph_search(problem, display=True)

# Extract the optimal path from the goal node.
optimal_path = [n.state for n in goal_node.path()]

In [ ]:
# Get the number of iterations
iterations = goal_node.path_cost

# Print the optimal path and other information.
print("Optimal Path:", optimal_path)
print("Number of Iterations:", iterations)
print(goal_node)

In [ ]:
def add_path(self, obstacles):
    for each in obstacles:
        print(each)
        x, y = each
        self.add_thing(Path(), (x, y))


add_path(forrest, optimal_path)

In [ ]:
forrest.run(0, delay=0.01)